# Database Query

This code is supposed to download the data from the DESI database daily and store relevant information in a fits file. For now I just query the data once and then put it in a fits file. From there, one could load the data, download all the data from the database and only add whats new on a daily basis.

In [1]:
from astropy.io import fits
from astropy.table import Table, hstack, vstack
from astropy import table

import numpy as np

import gasel as gs

from tqdm import tqdm
from astropy.io import ascii


In [2]:
# connect to db
cursor = gs.connect_to_time_db()

In [3]:
diameter = 4

### get the first data table

In [4]:
full_data = []

for x in tqdm(np.arange(0, 360, diameter)): # could improve this since 0 = 360... (was 370)
    #print("x: ", x)
    for y in np.arange(-20, 90, diameter): # (was 100)
        #print("x & y", x,y)
        #print(y)
        rows = gs.get_data_rad_search(x,y, diameter/2, cursor)
        #data = gs.make_list(rows)
        #print(rows[:2])
        if rows != None:
            #print(f"data {len(rows)}")
            #print(full_data)
            #print(data)
            #print(rows[:2])
            for obj in rows:
                
                full_data.append(obj)
        
full_table = Table(rows = full_data, names = gs.std_names)


full_table[:5]

100%|██████████| 90/90 [29:19<00:00, 19.55s/it]


TARGETID,TARGET_RA,TARGET_DEC,LASTNIGHT,Z,ZERR,ZWARN,FLUX_G,FLUX_R,FLUX_Z,SPECTYPE,BGS_TARGET,EBV,SERSIC
int64,float64,float64,int64,float64,float64,int64,float64,float64,float64,str6,int64,float64,float64
39627461389979471,0.199208396611172,-13.4284533588966,20211020,0.0842981615211128,1.02751859803205e-05,0,151.578369140625,349.176239013672,657.703186035156,GALAXY,131074,0.0286298077553511,1.75290405750275
39627461394171068,0.324188154725867,-13.3757445122443,20211020,0.358443849991424,6.26159169498368e-05,0,3.10386729240417,15.7543725967407,35.017635345459,GALAXY,131074,0.0287617538124323,4.0
39627461394173293,0.423061498719655,-13.3821980272798,20211020,7.96335573109056e-06,2.72051014406708e-06,0,25.3495140075684,58.3771476745605,91.9729995727539,STAR,0,0.0290689337998629,0.0
39627467278778997,0.54074701108748,-13.1672162663649,20211020,0.368523065341714,2.55883523529775e-05,0,5.02074289321899,15.623589515686,32.4087829589844,GALAXY,131074,0.0333173200488091,1.0
616088219582202155,0.626179874156068,-13.3202664780153,20211020,0.187326155709928,7.97282091423762e-06,1,0.0,0.0,0.0,GALAXY,0,0.0,0.0


In [5]:
len(full_data)

57480780

In [6]:
full_table.write('/global/homes/j/jgassert/full_table.fits', format='fits')

### get the second table

To do this: restart the kernel, otherwise RAM might be an issue!

In [ ]:
full_data2 = []

for x in tqdm(np.arange(2, 362, diameter)): # (was 360)
    #print("x: ", x)
    for y in np.arange(-18, 86, diameter): # (was 90)
        #print("x & y", x,y)
        #print(y)
        rows2 = gs.get_data_rad_search(x,y, diameter/2, cursor)
        #data = gs.make_list(rows)
        #print(rows[:2])
        if rows2 != None:
            #print(f"data {len(rows)}")
            #print(full_data)
            #print(data)
            #print(rows[:2])
            for obj2 in rows2:
                
                full_data2.append(obj2)

 77%|███████▋  | 69/90 [22:51<05:58, 17.07s/it]

In [ ]:
full_table2 = Table(rows = full_data2, names = gs.std_names)

In [ ]:
full_table2.write('/global/homes/j/jgassert/full_table2.fits', format='fits')

### read in data again and merge

*Note: I'd again restart the kernel, other RAM might become an issue!*

we can now read in both tables that we created earlier on and from there create our datasets

In [2]:
# get the data
full_table = Table.read("/global/homes/j/jgassert/full_table.fits")
full_table2 = Table.read("/global/homes/j/jgassert/full_table2.fits")

In [3]:
# create the new complete dataset
full_data = vstack([full_table, full_table2])

In [4]:
# now lets delete multiple entries
full_data = table.unique(full_data, keys = "TARGETID")

In [6]:
# now save this file on disk
full_data.write('/global/homes/j/jgassert/full_data.fits', format='fits', overwrite = False)

### Create good z data file

Good z data simply means deleting all entries with a redshift that has ZWARN != 0, i.e. there was a problem with the redshift estimation from redrock. I also cut out objects with a negative redshift, since that is unphysical.

In [ ]:
# cut out bad redshift objects as described above...
good_z_data = full_data[full_data["ZWARN"] == 0]
good_z_data = good_z_data[good_z_data["Z"] > 0]

In [ ]:
# save this data as well...
good_z_data.write('/global/homes/j/jgassert/good_z_data.fits', format='fits', overwrite = True)

In [ ]:
len(good_z_data), len(full_data)

(28207637, 45720036)

### Create BGS Bright file

To do so, I need the desimasks which is a bit cumbersome to import into my own python environment created with mamba. Therefore, open "CreateBGSBright.ipynb" and continue there.

# Everything below is "other stuff"

In [6]:
len(full_table2)

55860834

In [7]:
full_table2.write('/global/homes/j/jgassert/full_table2.ecsv', overwrite=True)  

In [ ]:
from astropy.io import ascii
full_table = ascii.read("/global/homes/j/jgassert/full_table.ecsv")

In [6]:
full_table[:5]

TARGETID,TARGET_RA,TARGET_DEC,LASTNIGHT,Z,ZERR,ZWARN,FLUX_G,FLUX_R,FLUX_Z,SPECTYPE,BGS_TARGET,EBV,SERSIC
int64,float64,float64,int64,float64,float64,int64,float64,float64,float64,str6,int64,float64,float64
39627461389979471,0.199208396611172,-13.4284533588966,20211020,0.0842981615211128,1.02751859803205e-05,0,349.176239013672,151.578369140625,657.703186035156,GALAXY,131074,0.0286298077553511,1.75290405750275
39627461394171068,0.324188154725867,-13.3757445122443,20211020,0.358443849991424,6.26159169498368e-05,0,15.7543725967407,3.10386729240417,35.017635345459,GALAXY,131074,0.0287617538124323,4.0
39627461394173293,0.423061498719655,-13.3821980272798,20211020,7.96335573109056e-06,2.72051014406708e-06,0,58.3771476745605,25.3495140075684,91.9729995727539,STAR,0,0.0290689337998629,0.0
39627467278778997,0.54074701108748,-13.1672162663649,20211020,0.368523065341714,2.55883523529775e-05,0,15.623589515686,5.02074289321899,32.4087829589844,GALAXY,131074,0.0333173200488091,1.0
616088219582202155,0.626179874156068,-13.3202664780153,20211020,0.187326155709928,7.97282091423762e-06,1,0.0,0.0,0.0,GALAXY,0,0.0,0.0


In [ ]:
full_table2 = ascii.read("/global/homes/j/jgassert/full_table2.ecsv") 

In [5]:
full_table2[:5]

TARGETID,TARGET_RA,TARGET_DEC,LASTNIGHT,Z,ZERR,ZWARN,FLUX_G,FLUX_R,FLUX_Z,SPECTYPE,BGS_TARGET,EBV,SERSIC
int64,float64,float64,int64,float64,float64,int64,float64,float64,float64,str6,int64,float64,float64
39627461389979471,0.199208396611172,-13.4284533588966,20211020,0.0842981615211128,1.02751859803205e-05,0,349.176239013672,151.578369140625,657.703186035156,GALAXY,131074,0.0286298077553511,1.75290405750275
39627461394171068,0.324188154725867,-13.3757445122443,20211020,0.358443849991424,6.26159169498368e-05,0,15.7543725967407,3.10386729240417,35.017635345459,GALAXY,131074,0.0287617538124323,4.0
39627461394173293,0.423061498719655,-13.3821980272798,20211020,7.96335573109056e-06,2.72051014406708e-06,0,58.3771476745605,25.3495140075684,91.9729995727539,STAR,0,0.0290689337998629,0.0
39627467278778997,0.54074701108748,-13.1672162663649,20211020,0.368523065341714,2.55883523529775e-05,0,15.623589515686,5.02074289321899,32.4087829589844,GALAXY,131074,0.0333173200488091,1.0
616088219582202155,0.626179874156068,-13.3202664780153,20211020,0.187326155709928,7.97282091423762e-06,1,0.0,0.0,0.0,GALAXY,0,0.0,0.0


In [ ]:
data = vstack([full_table, full_table2])

In [ ]:
data = table.unique(data, keys = "TARGETID")

In [ ]:
data

In [ ]:
# switch columns

In [11]:
data.write('/global/homes/j/jgassert/data.ecsv', overwrite=False)  

In [4]:
data = ascii.read("/global/homes/j/jgassert/data.ecsv") 

In [12]:
len(data)

44333797

In [6]:
data[:5]

TARGETID,TARGET_RA,TARGET_DEC,LASTNIGHT,Z,ZERR,ZWARN,FLUX_G,FLUX_R,FLUX_Z,SPECTYPE,BGS_TARGET,EBV,SERSIC
int64,float64,float64,int64,float64,float64,int64,float64,float64,float64,str6,int64,float64,float64
-833389526,227.433272279891,66.6519046819295,20230427,1.1824834644376,9.79136990918098e-05,5,0.0,0.0,0.0,GALAXY,0,0.0290863439440727,0.0
-833389518,228.204498084663,66.7769162408949,20230427,1.01705132530733,0.000198853226529452,5,0.0,0.0,0.0,GALAXY,0,0.0315619334578514,0.0
-833389510,227.642453626374,66.7226250371644,20230427,1.20045807712059,0.000130270720806434,5,0.0,0.0,0.0,GALAXY,0,0.0310118906199932,0.0
-833389492,227.755926256736,66.7768026589679,20230427,1.39280058555401,0.000191373523674881,2053,0.0,0.0,0.0,GALAXY,0,0.0316010527312756,0.0
-833389491,227.851676937586,66.7933443247276,20230427,0.747688478964211,8.9141824351558e-05,5,0.0,0.0,0.0,GALAXY,0,0.0321089513599873,0.0


In [13]:
good_z_data = data[data["ZWARN"] == 0]

In [14]:
good_z_data = good_z_data[good_z_data["Z"]>0]

In [15]:
good_z_data.write('/global/homes/j/jgassert/good_z_data.ecsv', overwrite=False)  

In [16]:
len(good_z_data)

27564701

In [17]:
good_r_flux_data = good_z_data[good_z_data["FLUX_R"] > 0]

The history saving thread hit an unexpected error (OperationalError('disk I/O error')).History will not be written to the database.


In [18]:
good_r_flux_data

TARGETID,TARGET_RA,TARGET_DEC,LASTNIGHT,Z,ZERR,ZWARN,FLUX_G,FLUX_R,FLUX_Z,SPECTYPE,BGS_TARGET,EBV,SERSIC
int64,float64,float64,int64,float64,float64,int64,float64,float64,float64,str6,int64,float64,float64
39627267193704468,87.2931346460236,-21.8854203931447,20210116,1.54139764931006,6.51899837787448e-05,0,0.678932547569275,0.544187486171722,0.814229965209961,GALAXY,0,0.0382782742381096,1.0
39627267193707272,87.3699284683912,-21.8766676366579,20210116,1.25698383007386,0.000243185890649753,0,1.8541316986084,1.37864375114441,1.71930611133575,QSO,0,0.0393409058451653,0.0
39627267197895900,87.4762538649076,-21.8806878993573,20210116,0.126248515944835,1.06512530797224e-05,0,0.420664072036743,0.235325276851654,0.483817219734192,GALAXY,0,0.0426067262887955,0.0
39627267197896032,87.4794499851022,-21.9167919737848,20210116,1.53241656944708,0.000234283455894663,0,0.668777704238892,0.317722111940384,0.890449166297913,QSO,0,0.0414619110524654,0.0
39627267197896327,87.4883036183932,-21.9417798804529,20210116,1.45163923277157,0.000133357679299795,0,1.55398964881897,0.562485635280609,3.03630256652832,QSO,0,0.0413365438580513,0.0
39627267197897458,87.5192385225095,-21.893491730542,20210116,0.998079051433133,8.48011440505117e-05,0,0.976543068885803,0.246809035539627,6.05792808532715,GALAXY,0,0.0431993380188942,4.0
39627267197898746,87.5568221296857,-21.887277112946,20210116,1.09801965558209,2.22775113331749e-05,0,0.301190555095673,0.246051579713821,0.458732664585114,GALAXY,0,0.0435711853206158,0.0
39627267197899514,87.5779409036305,-21.9212708133538,20210116,1.53353234507143,4.24849912504721e-05,0,0.871013402938843,0.807319462299347,1.12869954109192,GALAXY,0,0.0425399169325829,0.0
39627267197900264,87.600740028767,-21.8879578592919,20210116,0.629393988176992,0.000116359489808418,0,0.569007277488708,0.517570674419403,0.663229525089264,GALAXY,0,0.0432274788618088,0.0


In [22]:
good_r_flux_data.write('/global/homes/j/jgassert/good_r_flux_data.ecsv', overwrite=False)  

In [1]:
data = ascii.read("/global/homes/j/jgassert/good_z_data.ecsv") 

In [2]:
data[:5]

TARGETID,TARGET_RA,TARGET_DEC,LASTNIGHT,Z,ZERR,ZWARN,FLUX_G,FLUX_R,FLUX_Z,SPECTYPE,BGS_TARGET,EBV,SERSIC
int64,float64,float64,int64,float64,float64,int64,float64,float64,float64,str6,int64,float64,float64
-407206628,289.160148698913,63.8756777034092,20210513,0.488143245600312,6.00460085329576e-05,0,0.0,0.0,0.0,GALAXY,0,0.0,0.0
-407206517,289.24523373072,64.1182126208345,20210513,0.353953432359517,9.16563620937551e-05,0,0.0,0.0,0.0,GALAXY,0,0.0,0.0
-407206395,288.991137990503,64.4269390406163,20210513,1.04167986703249,0.000103598417102832,0,0.0,0.0,0.0,GALAXY,0,0.0,0.0
-407206306,288.328820102558,64.8171915439228,20210513,0.446085744563991,0.000103302897346752,0,0.0,0.0,0.0,GALAXY,0,0.0,0.0
-407206302,288.170414727353,64.8761951861955,20210513,5.11210896599966,0.000690663180059733,0,0.0,0.0,0.0,QSO,0,0.0,0.0


In [4]:
from desitarget.targets import desi_mask, bgs_mask, mws_mask

In [15]:
ii = data["BGS_TARGET"]&bgs_mask["BGS_BRIGHT_SOUTH"]
jj = data["BGS_TARGET"]&bgs_mask["BGS_BRIGHT_NORTH"]
kk = data["BGS_TARGET"]&bgs_mask["BGS_BRIGHT"]

In [43]:
bb = (data["BGS_TARGET"]&bgs_mask["BGS_BRIGHT_SOUTH"]) | (data["BGS_TARGET"]&bgs_mask["BGS_BRIGHT_NORTH"]) | (data["BGS_TARGET"]&bgs_mask["BGS_BRIGHT"])

In [46]:
sel = bb!= 0

In [44]:
bb[:5], np.min(bb), np.max(bb), np.mean(bb), np.unique(bb)

(<Column name='BGS_TARGET' dtype='int64' length=5>
 0
 0
 0
 0
 0,
 0,
 131074,
 23108.366947205413,
 <Column name='BGS_TARGET' dtype='int64' length=3>
      0
    514
 131074)

In [48]:
data[sel]

TARGETID,TARGET_RA,TARGET_DEC,LASTNIGHT,Z,ZERR,ZWARN,FLUX_G,FLUX_R,FLUX_Z,SPECTYPE,BGS_TARGET,EBV,SERSIC
int64,float64,float64,int64,float64,float64,int64,float64,float64,float64,str6,int64,float64,float64
39627322533350395,11.2017136545986,-19.3753250062112,20211215,0.274138421339174,7.3501930183854e-05,0,29.5323829650879,7.64354562759399,61.7763519287109,GALAXY,131074,0.0190095156431198,6.0
39627322533353907,11.3031570730046,-19.3778160129495,20211215,0.160812674805974,3.74070815089717e-05,0,44.3453521728516,21.8182907104492,74.1602554321289,GALAXY,131074,0.0181874949485064,1.12676537036896
39627322701120343,21.7265690067643,-19.404267970287,20211022,0.0347373193412903,3.04502283257839e-06,0,40.2380104064941,30.611120223999,47.7578353881836,GALAXY,131074,0.026496633887291,0.66505241394043
39627322701120643,21.7335960725857,-19.3756051133826,20211022,0.252243092187585,4.24386834432005e-05,0,69.4891662597656,19.3021106719971,144.997436523438,GALAXY,131074,0.0254172254353762,5.25417852401733
39627322701122007,21.7682640928066,-19.3895060286576,20211022,0.272183664497557,7.48448457991176e-06,0,19.1892528533936,10.1746435165405,29.1437644958496,GALAXY,131074,0.0247003585100174,0.544734835624695
39627322701123557,21.8145744614903,-19.3850046667042,20211022,0.376453438846267,8.36269523515822e-05,0,25.3339920043945,5.08830833435059,56.5602226257324,GALAXY,131074,0.0235172305256128,5.19685745239258
39627322701123768,21.8213999800153,-19.4174602359933,20211022,0.253063490932638,5.32955740966746e-05,0,58.5883979797363,16.9509563446045,112.311599731445,GALAXY,131074,0.0239106994122267,4.0
39627322701124950,21.8533345460488,-19.4169943574335,20211022,0.211212432373961,6.82018886333761e-05,0,37.117805480957,11.9251728057861,72.7160263061523,GALAXY,131074,0.0227459073066711,2.91189217567444
39627322701126385,21.8930127024219,-19.3979065422968,20211022,0.212754350303496,6.24986184945315e-05,0,27.8967342376709,8.75567245483398,57.0741996765137,GALAXY,131074,0.0210696440190077,3.03994846343994


In [51]:
data[sel].write('/global/homes/j/jgassert/good_z_data_BGSBright.ecsv', overwrite=False)  

In [9]:
import numpy as np

In [13]:
ii[:5], np.min(ii), np.max(ii), np.mean(ii), np.unique(ii)

(<Column name='BGS_TARGET' dtype='int64' length=5>
 0
 0
 0
 0
 0,
 0,
 0,
 0.0,
 <Column name='BGS_TARGET' dtype='int64' length=1>
 0)

In [16]:
jj[:5], np.min(jj), np.max(jj), np.mean(jj), np.unique(jj)

(<Column name='BGS_TARGET' dtype='int64' length=5>
 0
 0
 0
 0
 0,
 0,
 512,
 41.32786609947266,
 <Column name='BGS_TARGET' dtype='int64' length=2>
   0
 512)

In [23]:
selection = ii != 0 or jj != 0 or kk != 0


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [22]:
selection, np.min(selection), np.max(selection), np.mean(selection)

(array([False, False, False, ..., False, False, False]), False, False, 0.0)

In [25]:
data["BGS_TARGET"][:5], np.min(data["BGS_TARGET"]), np.max(data["BGS_TARGET"]), np.mean(data["BGS_TARGET"]), np.unique(data["BGS_TARGET"])

(<Column name='BGS_TARGET' dtype='int64' length=5>
 0
 0
 0
 0
 0,
 0,
 262148,
 28430.9616043722,
 <Column name='BGS_TARGET' dtype='int64' length=9>
      0
    257
    265
    514
   1028
  65537
  65545
 131074
 262148)

In [37]:
np.count_nonzero(data[data["BGS_TARGET"] == 514])

2224981

In [18]:
data = ascii.read("/global/homes/j/jgassert/full_table2.ecsv")

In [19]:
type(data)

astropy.table.table.Table

In [20]:
len(data)

55860834

In [21]:
data.rename_column('FLUX_G', 'FLUX_WRONG_G')
data.rename_column('FLUX_R', 'FLUX_WRONG_R')
data.rename_column('FLUX_WRONG_R', 'FLUX_G')
data.rename_column('FLUX_WRONG_G', 'FLUX_R')
data = data[gs.std_names]

In [2]:
data.write('full_table2.fits', format='fits')

NameError: name 'data' is not defined

In [5]:
data_corrected = Table.read('/global/homes/j/jgassert/full_table.fits', format='fits')

In [6]:
data_corrected2 = Table.read('/global/homes/j/jgassert/full_table2.fits', format='fits')

In [7]:
full_data = vstack([data_corrected, data_corrected2])

In [8]:
full_data = table.unique(full_data, keys = "TARGETID")

In [10]:
full_data.write('/global/homes/j/jgassert/full_data.fits', format='fits')

In [11]:
with fits.open('full_table.fits') as f:
    header = f[1].header

In [12]:
header

XTENSION= 'BINTABLE'           / binary table extension                         
BITPIX  =                    8 / array data type                                
NAXIS   =                    2 / number of array dimensions                     
NAXIS1  =                  110 / length of dimension 1                          
NAXIS2  =             56232189 / length of dimension 2                          
PCOUNT  =                    0 / number of group parameters                     
GCOUNT  =                    1 / number of groups                               
TFIELDS =                   14 / number of table fields                         
TTYPE1  = 'TARGETID'                                                            
TFORM1  = 'K       '                                                            
TTYPE2  = 'TARGET_RA'                                                           
TFORM2  = 'D       '                                                            
TTYPE3  = 'TARGET_DEC'      

In [9]:
data_corrected

TARGETID,TARGET_RA,TARGET_DEC,LASTNIGHT,Z,ZERR,ZWARN,FLUX_G,FLUX_R,FLUX_Z,SPECTYPE,BGS_TARGET,EBV,SERSIC
int64,float64,float64,int64,float64,float64,int64,float64,float64,float64,bytes6,int64,float64,float64
39627461389979471,0.199208396611172,-13.4284533588966,20211020,0.0842981615211128,1.02751859803205e-05,0,151.578369140625,349.176239013672,657.703186035156,GALAXY,131074,0.0286298077553511,1.75290405750275
39627461394171068,0.324188154725867,-13.3757445122443,20211020,0.358443849991424,6.26159169498368e-05,0,3.10386729240417,15.7543725967407,35.017635345459,GALAXY,131074,0.0287617538124323,4.0
39627461394173293,0.423061498719655,-13.3821980272798,20211020,7.96335573109056e-06,2.72051014406708e-06,0,25.3495140075684,58.3771476745605,91.9729995727539,STAR,0,0.0290689337998629,0.0
39627467278778997,0.54074701108748,-13.1672162663649,20211020,0.368523065341714,2.55883523529775e-05,0,5.02074289321899,15.623589515686,32.4087829589844,GALAXY,131074,0.0333173200488091,1.0
616088219582202155,0.626179874156068,-13.3202664780153,20211020,0.187326155709928,7.97282091423762e-06,1,0.0,0.0,0.0,GALAXY,0,0.0,0.0
616088219578008094,0.388241918672417,-13.2686674737008,20211020,0.522786405143744,6.49929837840397e-05,5,0.0,0.0,0.0,GALAXY,0,0.0,0.0
39627467274589294,0.504304210535291,-13.3632672858811,20211020,-0.000343635081887565,2.42554264343799e-05,0,15.5003890991211,23.754940032959,27.2029857635498,GALAXY,0,0.0301127638667822,0.0
39627467274588693,0.474659836366305,-13.1902772866147,20211020,0.000290135154212315,5.03837337123411e-06,0,10.2499370574951,29.6117382049561,53.6542320251465,STAR,0,0.0329338312149048,0.0
39627461389980420,0.250687965333069,-13.4330153264598,20211020,0.223159529486123,1.0147886100945e-05,0,2.66319489479065,13.4147672653198,29.86669921875,GALAXY,65537,0.0291027836501598,1.0


In [30]:
data[12345678:12345683]

TARGETID,TARGET_RA,TARGET_DEC,LASTNIGHT,Z,ZERR,ZWARN,FLUX_G,FLUX_R,FLUX_Z,SPECTYPE,BGS_TARGET,EBV,SERSIC
int64,float64,float64,int64,float64,float64,int64,float64,float64,float64,str6,int64,float64,float64
39628001171736132,342.379989052395,8.81943566360954,20221126,0.350454198245097,7.71357316282643e-05,0,13.1860857009888,2.93682980537415,28.1502208709717,GALAXY,65537,0.0704450756311417,1.0
39628001171736173,342.381854370417,8.78660864083436,20211126,0.268133278460806,2.74335846397887e-05,0,40.565258026123,12.7216529846191,88.1010513305664,GALAXY,131074,0.0726076290011406,6.0
39628001171736255,342.3857474762,8.73123744440379,20211126,0.0797433140535211,8.12130010714796e-06,0,11.3315420150757,6.00943040847778,17.9243297576904,GALAXY,65545,0.0763106644153595,1.0
39628001171736285,342.386792739222,8.66966201170546,20211126,0.400329630494547,9.5110411577271e-05,0,13.5422048568726,2.31780362129211,33.4780654907227,GALAXY,131074,0.0798639133572578,6.0
39628001171736294,342.387005336383,8.64240638407512,20211108,0.45987751026537,5.1939584708531e-05,0,16.9370861053467,3.01793622970581,44.5305671691895,GALAXY,131074,0.0806919112801552,5.41908073425293


In [31]:
data_corrected[12345678:12345683]

TARGETID,TARGET_RA,TARGET_DEC,LASTNIGHT,Z,ZERR,ZWARN,FLUX_G,FLUX_R,FLUX_Z,SPECTYPE,BGS_TARGET,EBV,SERSIC
int64,float64,float64,int64,float64,float64,int64,float64,float64,float64,bytes6,int64,float64,float64
39628001171736132,342.379989052395,8.81943566360954,20221126,0.350454198245097,7.71357316282643e-05,0,2.93682980537415,13.1860857009888,28.1502208709717,GALAXY,65537,0.0704450756311417,1.0
39628001171736173,342.381854370417,8.78660864083436,20211126,0.268133278460806,2.74335846397887e-05,0,12.7216529846191,40.565258026123,88.1010513305664,GALAXY,131074,0.0726076290011406,6.0
39628001171736255,342.3857474762,8.73123744440379,20211126,0.0797433140535211,8.12130010714796e-06,0,6.00943040847778,11.3315420150757,17.9243297576904,GALAXY,65545,0.0763106644153595,1.0
39628001171736285,342.386792739222,8.66966201170546,20211126,0.400329630494547,9.5110411577271e-05,0,2.31780362129211,13.5422048568726,33.4780654907227,GALAXY,131074,0.0798639133572578,6.0
39628001171736294,342.387005336383,8.64240638407512,20211108,0.45987751026537,5.1939584708531e-05,0,3.01793622970581,16.9370861053467,44.5305671691895,GALAXY,131074,0.0806919112801552,5.41908073425293


In [27]:
data_corrected[:5]

TARGETID,TARGET_RA,TARGET_DEC,LASTNIGHT,Z,ZERR,ZWARN,FLUX_G,FLUX_R,FLUX_Z,SPECTYPE,BGS_TARGET,EBV,SERSIC
int64,float64,float64,int64,float64,float64,int64,float64,float64,float64,bytes6,int64,float64,float64
39627322533350395,11.2017136545986,-19.3753250062112,20211215,0.274138421339174,7.3501930183854e-05,0,7.64354562759399,29.5323829650879,61.7763519287109,GALAXY,131074,0.0190095156431198,6.0
39627322533353907,11.3031570730046,-19.3778160129495,20211215,0.160812674805974,3.74070815089717e-05,0,21.8182907104492,44.3453521728516,74.1602554321289,GALAXY,131074,0.0181874949485064,1.12676537036896
39627322701120343,21.7265690067643,-19.404267970287,20211022,0.0347373193412903,3.04502283257839e-06,0,30.611120223999,40.2380104064941,47.7578353881836,GALAXY,131074,0.026496633887291,0.66505241394043
39627322701120643,21.7335960725857,-19.3756051133826,20211022,0.252243092187585,4.24386834432005e-05,0,19.3021106719971,69.4891662597656,144.997436523438,GALAXY,131074,0.0254172254353762,5.25417852401733
39627322701122007,21.7682640928066,-19.3895060286576,20211022,0.272183664497557,7.48448457991176e-06,0,10.1746435165405,19.1892528533936,29.1437644958496,GALAXY,131074,0.0247003585100174,0.544734835624695


In [42]:
data_corrected.write('good_z_data_BGSBright.fits', format='fits')

In [2]:
data_corrected = Table.read("good_z_data_BGSBright.fits")

In [3]:
data_corrected[:5]

TARGETID,TARGET_RA,TARGET_DEC,LASTNIGHT,Z,ZERR,ZWARN,FLUX_G,FLUX_R,FLUX_Z,SPECTYPE,BGS_TARGET,EBV,SERSIC
int64,float64,float64,int64,float64,float64,int64,float64,float64,float64,bytes6,int64,float64,float64
39627322533350395,11.2017136545986,-19.3753250062112,20211215,0.274138421339174,7.3501930183854e-05,0,7.64354562759399,29.5323829650879,61.7763519287109,GALAXY,131074,0.0190095156431198,6.0
39627322533353907,11.3031570730046,-19.3778160129495,20211215,0.160812674805974,3.74070815089717e-05,0,21.8182907104492,44.3453521728516,74.1602554321289,GALAXY,131074,0.0181874949485064,1.12676537036896
39627322701120343,21.7265690067643,-19.404267970287,20211022,0.0347373193412903,3.04502283257839e-06,0,30.611120223999,40.2380104064941,47.7578353881836,GALAXY,131074,0.026496633887291,0.66505241394043
39627322701120643,21.7335960725857,-19.3756051133826,20211022,0.252243092187585,4.24386834432005e-05,0,19.3021106719971,69.4891662597656,144.997436523438,GALAXY,131074,0.0254172254353762,5.25417852401733
39627322701122007,21.7682640928066,-19.3895060286576,20211022,0.272183664497557,7.48448457991176e-06,0,10.1746435165405,19.1892528533936,29.1437644958496,GALAXY,131074,0.0247003585100174,0.544734835624695
